In [4]:
from pynput.keyboard import Controller
import mediapipe as mp
import cv2
import numpy as np

In [11]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
pose = mp_pose.Pose(static_image_mode=False)

keyboard = Controller()

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ab = a - b
    cb = c - b

    dot_product = np.dot(ab, cb)
    magnitude_ab = np.linalg.norm(ab)
    magnitude_cb = np.linalg.norm(cb)

    angle = np.arccos(dot_product / (magnitude_ab * magnitude_cb))
    return np.degrees(angle)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks is not None:
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(128, 0, 250), thickness=2, circle_radius=3),
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
        )

        frame = cv2.flip(frame, 1)
        height, width, _ = frame.shape

        landmarks = results.pose_landmarks.landmark

        # Obtener puntos necesarios para hombros y manos
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW]
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]

        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
        left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]

        # Obtener puntos necesarios para las piernas
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]
        right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE]
        right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]

        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
        left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
        left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]

        # Coordenadas para hombros y manos
        right_shoulder_coords = (right_shoulder.x * width, right_shoulder.y * height)
        right_elbow_coords = (right_elbow.x * width, right_elbow.y * height)
        right_wrist_coords = (right_wrist.x * width, right_wrist.y * height)

        left_shoulder_coords = (left_shoulder.x * width, left_shoulder.y * height)
        left_elbow_coords = (left_elbow.x * width, left_elbow.y * height)
        left_wrist_coords = (left_wrist.x * width, left_wrist.y * height)

        # Coordenadas para piernas
        right_hip_coords = (right_hip.x * width, right_hip.y * height)
        right_knee_coords = (right_knee.x * width, right_knee.y * height)
        right_ankle_coords = (right_ankle.x * width, right_ankle.y * height)

        left_hip_coords = (left_hip.x * width, left_hip.y * height)
        left_knee_coords = (left_knee.x * width, left_knee.y * height)
        left_ankle_coords = (left_ankle.x * width, left_ankle.y * height)

        # Cálculo de ángulos para hombros y manos
        right_arm_angle = calculate_angle(right_shoulder_coords, right_elbow_coords, right_wrist_coords)
        left_arm_angle = calculate_angle(left_shoulder_coords, left_elbow_coords, left_wrist_coords)

        # Cálculo de ángulos para piernas
        right_leg_angle = calculate_angle(right_hip_coords, right_knee_coords, right_ankle_coords)
        left_leg_angle = calculate_angle(left_hip_coords, left_knee_coords, left_ankle_coords)

        # Mostrar ángulos en pantalla
        cv2.putText(frame, f"Angulo Brazo Derecho: {int(right_arm_angle)}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.putText(frame, f"Angulo Brazo Izquierdo: {int(left_arm_angle)}", (10, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2)
        cv2.putText(frame, f"Angulo Pierna Derecha: {int(right_leg_angle)}", (10, 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
        cv2.putText(frame, f"Angulo Pierna Izquierda: {int(left_leg_angle)}", (10, 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

        # Presionar "w" si ambos ángulos de brazos son menores o iguales a 45 grados
        # y si ambos ángulos de piernas son menores o iguales a 160 grados
        if right_arm_angle <= 45 or left_arm_angle <= 45:
            keyboard.press('w')
        else:
            keyboard.release('w')

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Presiona ESC para salir
        break

cap.release()
cv2.destroyAllWindows()
